In [1]:
# !pip install --no-deps --force-reinstall snowflake_ml_python-1.0.12-py3-none-any.whl

In [2]:
# !pip install peft transformers tokenizers vllm bitsandbytes datasets absl-py

### Import Libraries

In [3]:
from snowflake.snowpark.session import Session
from snowflake.ml.model.models import llm
from snowflake.ml.registry import model_registry
from snowflake.ml.model import deploy_platforms
import snowflake.ml.playground.finetune as ft_util
from snowflake.snowpark import VERSION
import snowflake.snowpark.functions as F

import sys
import os
import json
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Establish Secure Connection

In [4]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

User                        : DASH
Role                        : "DASH_SPCS"
Database                    : "DASH_DB"
Schema                      : "DASH_SCHEMA"
Warehouse                   : "DASH_L"
Snowflake version           : 7.40.0
Snowpark for Python version : 1.9.0


### Train Llama 2 in Snowpark Container Services

In [5]:
!mkdir merged_weights_dir
!mkdir output_weights_dir

mkdir: cannot create directory ‘merged_weights_dir’: File exists
mkdir: cannot create directory ‘output_weights_dir’: File exists


#### Load Training Dataset

In [6]:
# In the interest of time we will train and test on 10 records
train_df = pd.read_json("train.json", lines=True).head(10)
test_df = pd.read_json("train.json", lines=True).tail(10)

#### Train Llama 2

In [7]:
# Takes about ~3mins

merged_weights_dir = 'merged_weights_dir'
output_weights_dir = 'output_weights_dir'

runner = ft_util.FinetuneModelRunner(
    base_model_id="meta-llama/Llama-2-7b-chat-hf",
    task_type=ft_util.FinetuneTaskType.INSTRUCT,
    train_dataset=train_df,
    eval_dataset=train_df,
)

mcf = ft_util.ModelConfig(
    max_steps=1,
    quantization=ft_util.QuantizationConfig(
        k_bits=4,
    ),
    merged_weights_dir_path=merged_weights_dir,
)

_ = runner.train(output_weights_dir, model_cfg=mcf)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


Map (num_proc=8):   0%|          | 0/10 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/10 [00:00<?, ? examples/s]

Detected kernel version 5.4.181, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO 11-10 02:30:49 llm_engine.py:72] Initializing an LLM engine with config: model='merged_weights_dir', tokenizer='merged_weights_dir', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)
INFO 11-10 02:31:39 llm_engine.py:207] # GPU blocks: 864, # CPU blocks: 512


#### *Once the training completes, notice the "trainable params" and "all params" -- it should indicate that the total number of params are ~7billion; whereas we only trained using ~4million additional params*

#### *Also examine the contents of merged_weights_dir and output_weights_dir*

#### Evaluate Results on Training Dataset

*Examine the "output" and "predicted" columns*

In [8]:
eval_df = runner.get_eval_results()
eval_df

Processed prompts: 100%|██████████| 10/10 [00:04<00:00,  2.24it/s]


,instruction,input,output,language,predicted
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hello!\nfrosty: Well, hello! Who's spreading holiday cheer with me today?\ncaller: I'm Max from Sydney.\nfrosty: Hello, Max! Can you tell me what's on your wish list this holiday?\ncaller: Hmm, I am not sure. I guess I like cars.\nfrosty: We have a fun Bluey car. It's very cool. And also, there's a Teenage Mutant Ninja Turtles pizza delivery van! It's really fun.\ncaller: Oh, the bluey car sounds cool.\nfrosty: Great choice, Max! By the way, how do you plan to celebrate the holiday season with your family?\ncaller: We're going to the beach! It's summer here in Sydney.\nfrosty: Oh, that sounds wonderful, Max. So, we will put the Bluey car on your holiday wish list, okay?\ncaller: Yes, please!\nfrosty: It\u2019s all done. I hope your holiday is filled with joy and fun!""","{""toy_list"": [""Bluey Convertible and Figures""], ""location"": ""Sydney""}",EN,"Here is the JSON response with ""location"" and ""toy_list"" as keys:\n\n{\n""location"": ""Sydney"",\n""toy_list"": [\n""Bluey car"",\n""Teenage Mutant Ninja Turtles pizza delivery van""\n]\n}"
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi, Frosty\nfrosty: Hello there, bundle of joy! Can you share your name?\ncaller: I'm Lily.\nfrosty: Hello, Lily! And where are you calling from?\ncaller: I'm from London.\nfrosty: Beautiful London! Now, Lily, what are your two most wished toys this holiday season?\ncaller: I think I want the furby and... or maybe the transformers bumblebee. I can't decide.\nfrosty: Both are fascinating choices, Lily! Why are you interested in these toys?\ncaller: I like furby because it can talk, and transformers bumblebee because I like robots.\nfrosty: How about we put both on your wish list for now and you can decide later?\ncaller: Yes, let's do that.\nfrosty: Alright, Lily, both toys are now on your wish list. Enjoy your holiday and stay joyful!""","{""toy_list"": [""Furby interactive plush toy"", ""Transformers Rise of the Beasts Beast-Mode Bumblebee""], ""location"": ""London""}",EN,"Here is the JSON response with the requested keys:\n\n{\n""location"": ""London"",\n""toy_list"": [\n""furby"",\n""transformers bumblebee""\n]\n}"
2,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi Frosty! It's Benny here.\nfrosty: Hello, Benny! Nice to hear from you. Where are you calling from?\ncaller: From Auckland.\nfrosty: Beautiful Auckland! Now, tell Frosty your holiday wish, Benny.\ncaller: Can I have the ninja turtles delivery van?\nfrosty: Of course, you can. By the way, Benny, what do you enjoy the most about this time of the year?\ncaller: I like when we light up our house with beautiful lights.\nfrosty: That sounds like a sparkling time! So, I'll put the ninja turtles delivery van on your wish list. Does that sound good?\ncaller: Yes, that's perfect!\nfrosty: I wish you the best and super fun holidays, Benny!""","{""toy_list"": [""Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van""], ""location"": ""Auckland""}",EN,"Here is the JSON response with the requested keys:\n\n{\n""location"": ""Auckland"",\n""toy_list"": [\n""ninja turtles delivery van""\n]\n}"
3,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: Hello there! I'm Frosty, how can I add some sparkle to your holiday season?\ncaller: Hi Frosty! I'm Billy from Denver, United States.\nfrosty: Hi Billy! It's great to hear from you. What brings you to me today?\ncaller: Well I'm trying to decide whi

#### Inference on Custom Transcript

In [9]:
runner.infer(
     {
    'instruction': """
        "Extract JSON response with 'location' and 'toy_list' as keys. 'location': 
        Location of the caller. Include city only.'toy_list': List of toy names from the caller."
    """,
    'input': """
        "frosty: Hello, hello! Who's ready to chat with Frosty today?\ncaller: Hi, I'm Dustin from Cape Town, South Africa.\nfrosty: 
        Hi Dustin, it's awesome to meet you! Do you already know what you would like for your holiday gift?\ncaller: 
        Yeah, I think I want the ninja turtles delivery van, or maybe the robot dog. 
        I can't decide...\nfrosty: Both sound like entertaining choices! 
        Why are you interested in these particular toys?\ncaller: The ninja turtles van is so cool! And the robot dog, it's like a pet you don't have to clean up after.\nfrosty: 
        That's true! Savage choices, Dustin! How do you usually celebrate the holiday season?\ncaller: 
        We usually have a big family barbeque and play games all day long.\nfrosty: Sounds fun! What is your favorite thing about this time of year?
        \ncaller: I like that everyone is happy and I get to spend a lot of time with my family.
        \nfrosty: That's wonderful! So Dustin, have you thought any more about your holiday wish?
        \ncaller: I think I'm going to get the robot dog. It sounds fun and I won't have to walk it.
        \nfrosty: That's a clever choice, Dustin! I hope you have a fantastic holiday season!"
"""
    }
)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.58s/it]


'Here is the JSON response with the requested keys:\n\n{\n"location": "Cape Town",\n"toy_list": ["ninja turtles delivery van", "robot dog"]\n}'

#### Inference on Test Dataset
*Examine the "output" and "predicted" columns*

In [10]:
test_res = runner.infer_batch(test_df)
test_res

Processed prompts: 100%|██████████| 10/10 [00:04<00:00,  2.03it/s]


,instruction,input,output,language,predicted
980,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: Hello! Welcome to Frosty's Holiday Hotline. I'm Frosty, your festive voice bot. How may I help you today?\ncaller: Hi Frosty! I'm Ethan, and I need help picking a toy for the holidays.\nfrosty: Hi Ethan! I'd love to help you find a toy. What are your interests?\ncaller: I like superheroes and playing with action figures.\nfrosty: Awesome! How about Marvel's Spidey and His Amazing Friends Web Spinners playset or the Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van? Which one do you like more?\ncaller: I really like the Spiderman playset! Can you tell me more about it?\nfrosty: Sure! The Marvel's Spidey and His Amazing Friends Web Spinners playset features Spiderman and his friends, along with a web-spinning feature and various accessories. It's perfect for kids who love action figures and superheroes.\ncaller: That's perfect for me! Can you add that to my wish list?\nfrosty: Of course, Ethan! I have added Marvel's Spidey and His Amazing Friends Web Spinners playset to your wish list. Can you please tell me where you're calling from?\ncaller: I'm calling from Auckland, New Zealand.\nfrosty: Thank you, Ethan! Before we finish, can you share your favorite holiday memory with me?\ncaller: I remember when we went on a family trip to the mountains during the holidays. It was so much fun!\nfrosty: That sounds like a fantastic memory, Ethan. Thank you for calling Frosty's Holiday Hotline, and have an incredible holiday season!""","{""toy_list"": [""Marvel's Spidey and His Amazing Friends Web Spinners playset""], ""location"": ""Auckland""}",EN,"Here is the JSON response that meets the instruction:\n\n{\n""location"": ""Auckland, New Zealand"",\n""toy_list"": [\n""Marvel's Spidey and His Amazing Friends Web Spinners playset""\n]\n}"
981,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: Welcome to Frosty's Holiday Hotline! I'm Frosty, your fun-loving voice bot. How can I help you today?\ncaller: Hi Frosty! My name is Mia, and I want to add a toy to my holiday wish list.\nfrosty: Hi Mia! It's great to meet you. What type of toy are you interested in?\ncaller: I like playing with dolls and making my own stories.\nfrosty: That's fantastic, Mia! How about the Calico Critters Pony's Stylish Hair Salon or Gabbys Dollhouse Cruise Ship? Which one do you prefer?\ncaller: I think the Gabbys Dollhouse Cruise Ship sounds fun! Can you tell me more?\nfrosty: Sure! The Gabbys Dollhouse Cruise Ship is a beautifully designed playset featuring Gabby's favorite characters, along with a variety of accessories for creating exciting adventures on the high seas.\ncaller: That sounds perfect! I want to add that to my wish list.\nfrosty: Excellent choice, Mia! I have added the Gabbys Dollhouse Cruise Ship to your wish list. Where are you calling from?\ncaller: I'm calling from Johannesburg, South Africa.\nfrosty: Thank you, Mia! Before we go, can you tell me how you and your family plan to celebrate the holiday season?\ncaller: We usually go on a trip to visit our relatives and have a big family party. It's always lots of fun!\nfrosty: That sounds wonderful! Thank you for calling Frosty's Holiday Hotline, and have a fantastic holiday season!""","{""toy_list"": [""Gabbys Dollhouse Cruise Ship""], ""location"": ""Johannesburg""}",EN,"Here is the JSON response that extracts the ""location"" and ""toy_list"" keys from the input:\n```\n{\n ""location"": ""Johannesburg, South Africa"",\n ""toy_list"": [\n ""Gabbys Dollhouse Cruise Ship""\n ]\n}\n```"
982,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the ca

### Log and Deploy Fine-tuned Llama 2 

*NOTE: Notice that the "model_id_or_path" is set to the output weights folder where the weights from training the model are stored*

In [15]:
# NOTE: This will take ~20mins
MODEL_NAME = "LLAMA2_7b_CHAT"
MODEL_VERSION ="TunedV2.0"
MODEL_REGISTRY_DB = "DASH_DB"
MODEL_REGISTRY_SCHEMA = "DASH_SCHEMA"
COMPUTE_POOL = "DASH_BUILD_GPU3"
DEPLOYMENT_NAME = "fine_tuned_llama_predict"

options = llm.LLMOptions(
    token=connection_parameters["huggingface_token"],
    max_batch_size=100,
)

llama_model = llm.LLM(
    model_id_or_path=output_weights_dir,
    options=options
)

registry = model_registry.ModelRegistry(
    session=session, 
    database_name=MODEL_REGISTRY_DB, 
    schema_name=MODEL_REGISTRY_SCHEMA, 
    create_if_not_exists=True)

llama_model_ref = registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=llama_model
)

llama_model_ref.deploy(
    deployment_name=DEPLOYMENT_NAME, 
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True, 
    options={"compute_pool": COMPUTE_POOL, "num_gpus": 1})

RuntimeError: (1301) ResourceType.SERVICE DASH_DB.DASH_SCHEMA.service_e1e43f907f7711ee8545a6435ec35884 does not reach ready/done status.
Status: ResourceStatus.PENDING 


In [13]:
llama_model_ref = model_registry.ModelReference(registry=registry,model_name=MODEL_NAME,model_version=MODEL_VERSION)
df_predict_results = llama_model_ref.predict(deployment_name="llama_predict",data=test_df)
df_predict_results.show(max_width=600)